<font size="4"> <b> • DOMAIN: </b>Manufacturing</font>

<font size="4"> <b> • CONTEXT: </b>Company X curates and packages wine across various vineyards spread throughout the country

<font size="4"> <b> • DATA DESCRIPTION: </b>The data concerns the chemical composition of the wine and its respective quality.
    |
• Attribute Information:
1. A, B, C, D: specific chemical composition measure of the wine
2. Quality: quality of wine [ Low and High ]

<font size="4"> <b> • PROJECT OBJECTIVE: </b>Goal is to build a synthetic data generation model using the existing data provided by the company.

<b>Steps and tasks: [ Total Score: 5 points]</b>
    
1. Design a synthetic data generation model which can impute values [Attribute: Quality] wherever empty the company has missed recording the data.
</font>
 

<font size="5"><p style="color:black"> <b> 1. Design a synthetic data generation model which can impute values [Attribute: Quality] wherever empty the company has missed recording the data : </p></font>

<span style="font-family: Arial; font-weight:bold;font-size:1.6em;color:#0000FF;">1.1 Import the given datasets and explore shape and size.

In [211]:
company_data = pd.read_excel('Part2 - Company.xlsx') # Import the dataset
company_data.head()  # view the first 5 rows of the data

,A,B,C,D,Quality
0,47,27,45,108,Quality A
1,174,133,134,166,Quality B
2,159,163,135,131,NaN
3,61,23,3,44,Quality A
4,59,60,9,68,Quality A


In [212]:
company_data.size

305

In [213]:
company_data.shape

(61, 5)

In [214]:
company_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 61 entries, 0 to 60
Data columns (total 5 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   A        61 non-null     int64 
 1   B        61 non-null     int64 
 2   C        61 non-null     int64 
 3   D        61 non-null     int64 
 4   Quality  43 non-null     object
dtypes: int64(4), object(1)
memory usage: 2.5+ KB


<span style="font-family: Arial; font-weight:bold;font-size:1.6em;color:#0000FF;">1.2 Data cleansing:

<span style="font-family: Arial; font-weight:bold;font-size:1.3em;color:#00b3e5;">1.2.1 Checking for duplicates

In [215]:
# Identify duplicates records in the data
company_dupe = company_data.duplicated()
sum(company_dupe)

0

<span style="font-family: Arial; font-weight:bold;font-size:1.3em;color:#00b3e5;">1.2.2 Checking for duplicate index

In [216]:
company_dupe.index.is_unique

True

<span style="font-family: Arial; font-weight:bold;font-size:1.3em;color:#00b3e5;">1.2.3 Checking for missing values

In [217]:
def missing_check(df):
    total = df.isnull().sum().sort_values(ascending=False)   # total number of null values
    percent = (df.isnull().sum()/df.isnull().count()).sort_values(ascending=False)  # percentage of values that are null
    missing_data = pd.concat([total, percent], axis=1, keys=['Total', 'Percent'])  # putting the above two together
    return missing_data # return the dataframe
missing_check(company_data)

,Total,Percent
Quality,18,0.295082
A,0,0.000000
B,0,0.000000
C,0,0.000000
D,0,0.000000


* There are 18 missing values in the dataset in 'Quality' attribute

<span style="font-family: Arial; font-weight:bold;font-size:1.6em;color:#0000FF;">1.3 Data Preprocessing:

<span style="font-family: Arial; font-weight:bold;font-size:1.3em;color:#00b3e5;">1.3.1 Encoding categorical with integer values

In [218]:
company_data1 = company_data.copy()
#company_data1['Quality'] = company_data1['Quality'].replace({'Quality A': 0, 'Quality B': 1 })
company_data1

,A,B,C,D,Quality
0,47,27,45,108,Quality A
1,174,133,134,166,Quality B
2,159,163,135,131,NaN
3,61,23,3,44,Quality A
4,59,60,9,68,Quality A
...,...,...,...,...,...
56,200,186,185,179,Quality B
57,137,182,165,199,NaN
58,88,39,9,102,NaN
59,180,157,192,198,NaN


<span style="font-family: Arial; font-weight:bold;font-size:1.3em;color:#00b3e5;">1.3.2 Removing missing values from the original dataset and creating another dataset for missing values

In [219]:
missing_values = company_data1[company_data1['Quality'].isnull()]

In [220]:
company_data2 = company_data1.dropna()

<span style="font-family: Arial; font-weight:bold;font-size:1.3em;color:#00b3e5;">1.3.3 Predictor/Feature attribute and Target attribute

In [221]:
# Predictor/ Feature attribute
X = company_data2.drop('Quality', axis=1) 

# Target attribute
y = company_data2[['Quality']]

<span style="font-family: Arial; font-weight:bold;font-size:1.3em;color:#00b3e5;">1.3.4 Min-Max Scaling

In [222]:
X = (X - np.min(X))/(np.max(X)-np.min(X))

<span style="font-family: Arial; font-weight:bold;font-size:1.3em;color:#00b3e5;">1.3.5 Checking for Multicolinearity

In [223]:
# Calculate the VIFs for the new model
from statsmodels.stats.outliers_influence import variance_inflation_factor

vif = pd.DataFrame()
X1 = X
vif['Features'] = X1.columns
vif['VIF'] = [variance_inflation_factor(X1.values, i) for i in range(X1.shape[1])]
vif['VIF'] = round(vif['VIF'], 2)
vif = vif.sort_values(by = "VIF", ascending = False)
vif

,Features,VIF
2,C,15.07
1,B,12.31
3,D,10.96
0,A,10.58


* Since all values are in similar range, they're all acceptable for modelling

<span style="font-family: Arial; font-weight:bold;font-size:1.6em;color:#0000FF;">1.4 Logistic Regression model

In [224]:
X_train = X
y_train = y
X_test = missing_values.drop('Quality',axis = 1)

In [225]:
X_train.shape

(43, 4)

In [226]:
y_train.shape

(43, 1)

In [227]:
X_test.shape

(18, 4)

In [228]:
from sklearn.linear_model import LogisticRegression
LogReg = LogisticRegression()
LogReg.fit(X_train, y_train)
y_pred = LogReg.predict(X_test)

missing_values['Quality'] = missing_values.Quality
final_predicted_dataset = company_data1.copy()
final_predicted_dataset.loc[(final_predicted_dataset.Quality.isnull()), 'Quality'] = y_pred
final_predicted_dataset

D:\Anaconda\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
<ipython-input-228-bbc3aa171096>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  missing_values['Quality'] = missing_values.Quality


,A,B,C,D,Quality
0,47,27,45,108,Quality A
1,174,133,134,166,Quality B
2,159,163,135,131,Quality B
3,61,23,3,44,Quality A
4,59,60,9,68,Quality A
...,...,...,...,...,...
56,200,186,185,179,Quality B
57,137,182,165,199,Quality B
58,88,39,9,102,Quality B
59,180,157,192,198,Quality B


In [229]:
final_predicted_dataset[final_predicted_dataset['Quality']=='Quality A'].value_counts().sum()

26

In [230]:
final_predicted_dataset[final_predicted_dataset['Quality']=='Quality B'].value_counts().sum()

35

In [231]:
np.sum(26+35)

61

In [232]:
final_predicted_dataset.shape

(61, 5)

In [233]:
#final_predicted_dataset.to_excel('final_predicted_dataset.xlsx')